<a href="https://colab.research.google.com/github/ykitaguchi77/CorneAI/blob/main/50_knock_csv_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**「前原の100本ノック」クラス枚数確認**

In [23]:
from google.colab import drive
drive.mount("/content/drive")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
import pandas as pd

# 100問_正解リスト.csvファイルからデータフレームを作成する
ans_df = pd.read_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト.csv')

# 100問ノック_スマホと細隙灯区分.csvファイルからデータフレームを作成する
knock_df = pd.read_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問ノック_スマホと細隙灯区分.csv')

In [ ]:
ans_df

In [ ]:
knock_df

In [ ]:
import pandas as pd

# "device"の行をdf_answerに追加
ans_df['device'] = ""

# "slit"と"smartphone"の列を取得
slit = knock_df["slit"].tolist()
smartphone = knock_df["smartphone"].tolist()

# "device"列に"slit"と"smartphone"を入力
for i in range(1,101):
    if i in slit:
        ans_df.loc[ans_df["number"]==i, "device"] = "slit"
    elif i in smartphone:
        ans_df.loc[ans_df["number"]==i, "device"] = "smartphone"

# インデックスをリセット
ans_df = ans_df.reset_index(drop=True)
ans_df

In [ ]:
smartphone_df = ans_df[ans_df['device'] == 'smartphone']
smartphone_classification_counts = smartphone_df['classification'].value_counts()
print("前原100本ノック_スマホ画像の各クラス枚数一覧")
print(smartphone_classification_counts)

In [ ]:
slit_df = ans_df[ans_df['device'] == 'slit']
slit_classification_counts = slit_df['classification'].value_counts()
print("前原100本ノック_スリット画像の各クラス枚数一覧")
print(slit_classification_counts)

In [ ]:
ans_df


In [55]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 150)

In [ ]:
ans_df = ans_df.replace("腫瘍", "tumor")
ans_df = ans_df.replace("瘢痕", "scar")
ans_df = ans_df.replace("水晶体混濁", "lens-opacity")
ans_df = ans_df.replace("緑内障発作", "APAC")
ans_df = ans_df.replace("感染", "infection")
ans_df = ans_df.replace("非感染", "non-infection")
ans_df = ans_df.replace("水疱性角膜症", "bullous")
ans_df = ans_df.replace("沈着", "deposit")
ans_df = ans_df.replace("正常", "normal")
ans_df


In [ ]:
ans_df.to_csv("/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト_2.csv")

In [ ]:
ans_df = pd.read_csv("/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/100問_正解リスト_2.csv")

#**CorneAI_computerの解析**

In [57]:
# 100問_正解リスト.csvファイルからデータフレームを作成する
PC_df = pd.read_csv('/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/北口先生へ100問corneAIと正解の選択肢.csv')

In [58]:
# クラスを統一されたものに変更する

# 置き換え用の辞書を定義する
replace_dict = {
    '腫瘍性病変（翼状片、扁平上皮癌、デルモイドなど）': 'tumor',
    '瘢痕': 'scar',
    '水晶体混濁（白内障、眼内レンズ眼の嚢混濁や膨隆皮質）': 'lens-opacity',
    '急性緑内障発作': 'APAC',
    '感染性角膜炎（細菌、真菌、上皮型ヘルペス、アメーバ）': 'infection',
    '正常': 'normal',
    '沈着、ジストロフィ（顆粒状、斑状、帯状、格子状など）': 'deposit',
    '非感染性角膜炎（モーレン、周辺部潰瘍、カタル性、フリクテンなど）': 'non-infection',
    '水疱性角膜症': 'bullous'
}

# 置換処理を行う関数を定義する
def replace_text(text):
    if isinstance(text, float):
        return ''
    for key, value in replace_dict.items():
        if key in text:
            return value
    return text

# corneaAIカラムの値を置換する
PC_df['corneaAI'] = PC_df['corneaAI'].apply(lambda x: replace_text(x))

# 正解選択肢カラムの値を置換する
PC_df['正解選択肢'] = PC_df['正解選択肢'].apply(lambda x: replace_text(x) if x else '')

PC_df['device'] = ans_df['device']

In [59]:
# Computerの正答率を計算
num_correct = sum(PC_df['corneaAI'] == PC_df['正解選択肢'])
total_images = len(PC_df)
prob_correct = num_correct / total_images

# deviceがslitのデータで正解率を計算する
slit_df = PC_df[PC_df['device'] == 'slit']
num_correct_slit = sum(slit_df['corneaAI'] == slit_df['正解選択肢'])
total_images_slit = len(slit_df)
prob_correct_slit = num_correct_slit / total_images_slit

# deviceがsmartphoneのデータで正解率を計算する
smartphone_df = PC_df[PC_df['device'] == 'smartphone']
num_correct_smartphone = sum(smartphone_df['corneaAI'] == smartphone_df['正解選択肢'])
total_images_smartphone = len(smartphone_df)
prob_correct_smartphone = num_correct_smartphone / total_images_smartphone

# 結果を出力する
print(f"Computerと正解選択肢が一致する確率は{prob_correct:.2f} ({num_correct}/{total_images})です。")
print(f"deviceがslitのときの正解率は{prob_correct_slit:.2f}({num_correct_slit}/{total_images_slit})です。")
print(f"deviceがsmartphoneのときの正解率は{prob_correct_smartphone:.2f}({num_correct_smartphone}/{total_images_smartphone})です。")

Computerと正解選択肢が一致する確率は0.86 (86/100)です。
deviceがslitのときの正解率は0.88(44/50)です。
deviceがsmartphoneのときの正解率は0.84(42/50)です。


#**Computer vs Smartphone**

In [ ]:
import os
import numpy as np
from sklearn.metrics import cohen_kappa_score


pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 150)

parent_dir = "/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問"
os.chdir(parent_dir)
os.listdir(parent_dir)



In [ ]:
# top1の確率、各top3のindexを出力する

def top3(data_df, row):
    # smartphoneの判定の列を取り出す
    cols = data_df.iloc[row, 2:11].astype(float).fillna(0)

    # numpy.argsortを使って、各列の値が降順になるようにインデックスを並び替える
    idx = np.argsort(-cols)

    # インデックスから列名を取得する
    names = cols.index[idx]

    # top1の確率を取得する
    top_prob = cols[names[0]]

    # top3の列名とtop1の確率をリストとして返す
    return [top_prob, names[0], names[1], names[2]]

def read_csv(path):
    df = pd.read_csv(path, encoding='cp932')
    df_top3 = pd.DataFrame(index=[], columns=[])
    df_top3["number"] = PC_df["number"]
    for row in range(len(df)):
        df_top3.loc[row, ["top1_prob", "top1", "top2", "top3"]] = top3(df, row)
    df_top3["computer"] = PC_df["corneaAI"]
    df_top3["answer"] = PC_df["正解選択肢"]
    df_top3["device"] = PC_df["device"]
    return df_top3



def show_accuracy(df, name):
    # answer == top1である確率を計算する
    top1_accuracy = sum(df.apply(lambda row: row['answer'] in [row['top1']], axis=1)) / len(df) * 100
    print(f'答えがtop1と一致する確率: {top1_accuracy:.2f}% ({sum(df["answer"] == df["top1"])}/{len(df)})')

    # 答えがtop1, top2, またはtop3である確率を計算
    top3_accuracy = sum(df.apply(lambda row: row['answer'] in [row['top1'], row['top2'], row['top3']], axis=1)) / len(df) * 100
    print("答えがtop1, top2, またはtop3と一致する確率：{:.2f}% ({}/{})".format(top3_accuracy, sum(df.apply(lambda row: row['answer'] in [row['top1'], row['top2'], row['top3']], axis=1)), len(df)))

    top1_computer = sum(df.apply(lambda row: row['computer'] in [row['top1']], axis=1)) / len(df) * 100
    print(f'computerの判定がtop1と一致する確率: {top1_computer:.2f}% ({sum(df["computer"] == df["top1"])}/{len(df)})')

    cohens_kappa = cohen_kappa_score(df['top1'], df['computer'])
    print(f"cohen's kappa: {cohens_kappa:.2f}")

    top3_computer = sum(df.apply(lambda row: row['computer'] in [row['top1'], row['top2'], row['top3']], axis=1)) / len(df) * 100
    print("computerの判定がtop1, top2, またはtop3と一致する確率：{:.2f}% ({}/{})".format(top3_computer, sum(df.apply(lambda row: row['computer'] in [row['top1'], row['top2'], row['top3']], axis=1)), len(df)))


    output1 = [top1_accuracy, top3_accuracy, top1_computer, cohens_kappa, top3_computer]


    df_copy = df

    # 尤度90以上に限定
    df = df_copy[df_copy['top1_prob'] >= 90]

    # answer == top1である確率を計算する
    top1_accuracy = sum(df.apply(lambda row: row['answer'] in [row['top1']], axis=1)) / len(df) * 100
    print(f'尤度90以上で答えがtop1と一致する確率: {top1_accuracy:.2f}% ({sum(df["answer"] == df["top1"])}/{len(df)})')

    # 答えがtop1, top2, またはtop3である確率を計算
    top3_accuracy = sum(df.apply(lambda row: row['answer'] in [row['top1'], row['top2'], row['top3']], axis=1)) / len(df) * 100
    print("尤度90以上で答えがtop1, top2, またはtop3と一致する確率：{:.2f}% ({}/{})".format(top3_accuracy, sum(df.apply(lambda row: row['answer'] in [row['top1'], row['top2'], row['top3']], axis=1)), len(df)))

    top1_computer = sum(df.apply(lambda row: row['computer'] in [row['top1']], axis=1)) / len(df) * 100
    print(f'尤度90以上でcomputerの判定がtop1と一致する確率: {top1_computer:.2f}% ({sum(df["computer"] == df["top1"])}/{len(df)})')

    top3_computer = sum(df.apply(lambda row: row['computer'] in [row['top1'], row['top2'], row['top3']], axis=1)) / len(df) * 100
    print("尤度90以上でcomputerの判定がtop1, top2, またはtop3と一致する確率：{:.2f}% ({}/{})".format(top3_computer, sum(df.apply(lambda row: row['computer'] in [row['top1'], row['top2'], row['top3']], axis=1)), len(df)))

    output2 = [top1_accuracy, top3_accuracy, top1_computer, top3_computer]
    output = output1+output2


    # 尤度90未満に限定
    df = df_copy[df_copy['top1_prob'] < 90]

    # answer == top1である確率を計算する
    top1_accuracy = sum(df.apply(lambda row: row['answer'] in [row['top1']], axis=1)) / len(df) * 100
    print(f'尤度90未満で答えがtop1と一致する確率: {top1_accuracy:.2f}% ({sum(df["answer"] == df["top1"])}/{len(df)})')

    # 答えがtop1, top2, またはtop3である確率を計算
    top3_accuracy = sum(df.apply(lambda row: row['answer'] in [row['top1'], row['top2'], row['top3']], axis=1)) / len(df) * 100
    print("尤度90未満で答えがtop1, top2, またはtop3と一致する確率：{:.2f}% ({}/{})".format(top3_accuracy, sum(df.apply(lambda row: row['answer'] in [row['top1'], row['top2'], row['top3']], axis=1)), len(df)))

    top1_computer = sum(df.apply(lambda row: row['computer'] in [row['top1']], axis=1)) / len(df) * 100
    print(f'尤度90未満でcomputerの判定がtop1と一致する確率: {top1_computer:.2f}% ({sum(df["computer"] == df["top1"])}/{len(df)})')

    top3_computer = sum(df.apply(lambda row: row['computer'] in [row['top1'], row['top2'], row['top3']], axis=1)) / len(df) * 100
    print("尤度90未満でcomputerの判定がtop1, top2, またはtop3と一致する確率：{:.2f}% ({}/{})".format(top3_computer, sum(df.apply(lambda row: row['computer'] in [row['top1'], row['top2'], row['top3']], axis=1)), len(df)))

    output2 = [top1_accuracy, top3_accuracy, top1_computer, top3_computer]
    output = output1+output2

    print("")
    print("")

    return output




def calculate_accuracy(df, name):
    df_all = df
    df_smartphone = df[df["device"] == "smartphone"]
    df_slit = df[df["device"] == "slit"]

    output = []

    print("all")
    output.extend(show_accuracy(df, name))

    print("smartphone")
    output.extend(show_accuracy(df_smartphone, name))

    print("slit")
    output.extend(show_accuracy(df_slit, name))

    return output



In [61]:
# 結果をまとめるためのdataframeを作成
base = ["top1_accuracy", "top3_accuracy", "top1_computer", "cohen's kappa", "top3_computer", "likelihood90_top1_accuracy", "likelihood90_top3_accuracy", "likelihood90_top1_computer", "likelihood90_top3_computer"]
all = ["all_" + item for item in base]
smartphone = ["smartphone_" + item for item in base]
slit = ["slit_" + item for item in base]
index = all+smartphone+slit

df_result = pd.DataFrame(index=index, columns=[])

In [62]:
monitor_list = ["PC_vs_smartphone_thinkpad1.csv", "PC_vs_smartphone_macbookpro1.csv", 'PC_vs_smartphone_karute_kitaguchi.csv', 'PC_vs_smartphone_omen.csv']

df_0 = read_csv(monitor_list[0])
df_1 = read_csv(monitor_list[1])
df_2 = read_csv(monitor_list[2])
df_3 = read_csv(monitor_list[3])

print("thinkpad")
output = calculate_accuracy(df_0, "thinkpad")
df_result["thinkpad"] = output

print("macbookpro")
output = calculate_accuracy(df_1, "macbookpro")
df_result["macbookpro"] = output

print("eizo")
output = calculate_accuracy(df_1, "eizo")
df_result["eizo"] = output

print("omen")
output = calculate_accuracy(df_3, "omen")
df_result["omen"] = output

thinkpad
all
答えがtop1と一致する確率: 81.00% (81/100)
答えがtop1, top2, またはtop3と一致する確率：93.00% (93/100)
computerの判定がtop1と一致する確率: 83.00% (83/100)
cohen's kappa: 0.81
computerの判定がtop1, top2, またはtop3と一致する確率：94.00% (94/100)
尤度90以上で答えがtop1と一致する確率: 88.57% (62/70)
尤度90以上で答えがtop1, top2, またはtop3と一致する確率：91.43% (64/70)
尤度90以上でcomputerの判定がtop1と一致する確率: 90.00% (63/70)
尤度90以上でcomputerの判定がtop1, top2, またはtop3と一致する確率：92.86% (65/70)
尤度90未満で答えがtop1と一致する確率: 63.33% (19/30)
尤度90未満で答えがtop1, top2, またはtop3と一致する確率：96.67% (29/30)
尤度90未満でcomputerの判定がtop1と一致する確率: 66.67% (20/30)
尤度90未満でcomputerの判定がtop1, top2, またはtop3と一致する確率：96.67% (29/30)


smartphone
答えがtop1と一致する確率: 72.00% (36/50)
答えがtop1, top2, またはtop3と一致する確率：88.00% (44/50)
computerの判定がtop1と一致する確率: 80.00% (40/50)
cohen's kappa: 0.77
computerの判定がtop1, top2, またはtop3と一致する確率：92.00% (46/50)
尤度90以上で答えがtop1と一致する確率: 80.00% (24/30)
尤度90以上で答えがtop1, top2, またはtop3と一致する確率：83.33% (25/30)
尤度90以上でcomputerの判定がtop1と一致する確率: 86.67% (26/30)
尤度90以上でcomputerの判定がtop1, top2, またはtop3と一致する確率：90.00% (27/

In [ ]:
df_0


In [ ]:
repeatability_list = ["PC_vs_smartphone_macbookpro1.csv", "PC_vs_smartphone_macbookpro2.csv", 'PC_vs_smartphone_macbookpro3.csv']

df_0 = read_csv(repeatability_list[0])
df_1 = read_csv(repeatability_list[1])
df_2 = read_csv(repeatability_list[2])

print("macbookpro1")
output = calculate_accuracy(df_0, "1")
df_result["macbookpro1"] = output
print("macbookpro2")
output = calculate_accuracy(df_1, "2")
df_result["macbookpro2"] = output
print("macbookpro3")
output = calculate_accuracy(df_2, "3")
df_result["macbookpro3"] = output

macbookpro1
all
答えがtop1と一致する確率: 79.00% (79/100)
答えがtop1, top2, またはtop3と一致する確率：92.00% (92/100)
computerの判定がtop1と一致する確率: 86.00% (86/100)
cohen's kappa: 0.84
computerの判定がtop1, top2, またはtop3と一致する確率：97.00% (97/100)
尤度90以上で答えがtop1と一致する確率: 91.04% (61/67)
尤度90以上で答えがtop1, top2, またはtop3と一致する確率：95.52% (64/67)
尤度90以上でcomputerの判定がtop1と一致する確率: 95.52% (64/67)
尤度90以上でcomputerの判定がtop1, top2, またはtop3と一致する確率：98.51% (66/67)
尤度90未満で答えがtop1と一致する確率: 54.55% (18/33)
尤度90未満で答えがtop1, top2, またはtop3と一致する確率：84.85% (28/33)
尤度90未満でcomputerの判定がtop1と一致する確率: 66.67% (22/33)
尤度90未満でcomputerの判定がtop1, top2, またはtop3と一致する確率：93.94% (31/33)


smartphone
答えがtop1と一致する確率: 70.00% (35/50)
答えがtop1, top2, またはtop3と一致する確率：86.00% (43/50)
computerの判定がtop1と一致する確率: 84.00% (42/50)
cohen's kappa: 0.82
computerの判定がtop1, top2, またはtop3と一致する確率：96.00% (48/50)
尤度90以上で答えがtop1と一致する確率: 83.33% (25/30)
尤度90以上で答えがtop1, top2, またはtop3と一致する確率：90.00% (27/30)
尤度90以上でcomputerの判定がtop1と一致する確率: 96.67% (29/30)
尤度90以上でcomputerの判定がtop1, top2, またはtop3と一致する確率：100.00% 

In [63]:
# macbookpro使用、iPhone11pro, iPhone13pro, iPhone14pro
iPhone_list = ["PC_vs_smartphone_iPhone11.csv", 'PC_vs_smartphone_macbookpro1.csv', "PC_vs_smartphone_iPhone14.csv"]

df_0 = read_csv(iPhone_list[0])
df_1 = read_csv(iPhone_list[1])
df_2 = read_csv(iPhone_list[2])

print("iPhone11")
output = calculate_accuracy(df_0, "iphone11")
df_result["iphone11"] = output
print("iPhone13")
output = calculate_accuracy(df_1, "iphone13")
df_result["iphone13"] = output
print("iPhone14")
output = calculate_accuracy(df_2, "iphone14")
df_result["iphone14"] = output

iPhone11
all
答えがtop1と一致する確率: 79.00% (79/100)
答えがtop1, top2, またはtop3と一致する確率：95.00% (95/100)
computerの判定がtop1と一致する確率: 84.00% (84/100)
cohen's kappa: 0.82
computerの判定がtop1, top2, またはtop3と一致する確率：98.00% (98/100)
尤度90以上で答えがtop1と一致する確率: 88.57% (62/70)
尤度90以上で答えがtop1, top2, またはtop3と一致する確率：97.14% (68/70)
尤度90以上でcomputerの判定がtop1と一致する確率: 92.86% (65/70)
尤度90以上でcomputerの判定がtop1, top2, またはtop3と一致する確率：98.57% (69/70)
尤度90未満で答えがtop1と一致する確率: 56.67% (17/30)
尤度90未満で答えがtop1, top2, またはtop3と一致する確率：90.00% (27/30)
尤度90未満でcomputerの判定がtop1と一致する確率: 63.33% (19/30)
尤度90未満でcomputerの判定がtop1, top2, またはtop3と一致する確率：96.67% (29/30)


smartphone
答えがtop1と一致する確率: 72.00% (36/50)
答えがtop1, top2, またはtop3と一致する確率：92.00% (46/50)
computerの判定がtop1と一致する確率: 82.00% (41/50)
cohen's kappa: 0.79
computerの判定がtop1, top2, またはtop3と一致する確率：100.00% (50/50)
尤度90以上で答えがtop1と一致する確率: 84.38% (27/32)
尤度90以上で答えがtop1, top2, またはtop3と一致する確率：93.75% (30/32)
尤度90以上でcomputerの判定がtop1と一致する確率: 90.62% (29/32)
尤度90以上でcomputerの判定がtop1, top2, またはtop3と一致する確率：100.00% (3

In [ ]:
df_result = df_result.round(2)
df_result
df_result.to_csv("/content/drive/MyDrive/研究/進行中の研究/角膜スマートフォンAIプロジェクト/前原の100問/result_accuacy.csv")


# **Calculate Fleiss Kappa**

In [64]:
df_monitor = pd.DataFrame(index=[], columns=[])
df_monitor["Answer"] = PC_df["正解選択肢"]
df_monitor["PC"] = PC_df["corneaAI"]
df_monitor["df_0"] = df_0["top1"]
df_monitor["df_1"] = df_1["top1"]
df_monitor["df_2"] = df_2["top1"]
#df_monitor["df_3"] = df_3["top1"]
df_monitor["device"] = df_0["device"]

In [65]:
df_monitor

,Answer,PC,df_0,df_1,df_2,device
0,tumor,tumor,tumor,tumor,tumor,slit
1,scar,scar,scar,scar,scar,slit
2,lens-opacity,lens-opacity,scar,scar,scar,smartphone
3,APAC,APAC,APAC,APAC,APAC,smartphone
4,infection,infection,infection,infection,infection,slit
5,infection,infection,infection,infection,infection,smartphone
6,lens-opacity,lens-opacity,normal,lens-opacity,lens-opacity,slit
7,normal,normal,normal,normal,normal,slit
8,normal,normal,normal,normal,normal,smartphone
9,scar,scar,scar,scar,scar,smartphone


In [66]:
from statsmodels.stats.inter_rater import fleiss_kappa

def calculate_fleiss_kappa (df_monitor):
    categories = {
        'tumor': 0,
        'scar': 1,
        'lens-opacity': 2,
        'APAC': 3,
        'infection': 4,
        'normal': 5,
        'deposit': 6,
        'non-infection': 7,
        'bullous': 8
    }

    try:
        df_monitor = df_monitor.drop(["Answer", "PC", "device"], axis=1).replace(categories)
        print(df_monitor)
    except:
        pass

    # 同意度の表を作成
    n_categories = 10
    table = np.zeros((len(df_monitor.index), n_categories))
    for i, row in df_monitor.iterrows():
        for j in range(n_categories):
            table[i, j] = sum(row == j)

    # Fleiss' Kappa の計算
    kappa = fleiss_kappa(table, method='fleiss')
    print('Fleiss\' Kappa: {:.3f}'.format(kappa))



    # # 全て一致しているものの割合を計算
    # all_match_count = len(df_monitor[df_monitor['df_0'] == df_monitor['df_1']][df_monitor['df_1'] == df_monitor['df_2']])
    # all_match_ratio = all_match_count / len(df_monitor)

    # # 2つだけ一致しているものの割合を計算
    # two_match_count = len(df_monitor[df_monitor['df_0'] == df_monitor['df_1']][df_monitor['df_1'] != df_monitor['df_2']]) + len(df_monitor[df_monitor['df_0'] == df_monitor['df_2']][df_monitor['df_0'] != df_monitor['df_1']]) + len(df_monitor[df_monitor['df_1'] == df_monitor['df_2']][df_monitor['df_0'] != df_monitor['df_1']])
    # two_match_ratio = two_match_count / len(df_monitor)

    # # 全て異なっているものの割合を計算
    # all_different_count = len(df_monitor[df_monitor['df_0'] != df_monitor['df_1']][df_monitor['df_1'] != df_monitor['df_2']])
    # all_different_ratio = all_different_count / len(df_monitor)

    # # 結果の表示
    # print("全て一致しているものの割合:", all_match_ratio)
    # print("2つだけ一致しているものの割合:", two_match_ratio)
    # print("全て異なっているものの割合:", all_different_ratio)


In [ ]:
df_monitor

In [67]:
calculate_fleiss_kappa(df_monitor)

    df_0  df_1  df_2
0      0     0     0
1      1     1     1
2      1     1     1
3      3     3     3
4      4     4     4
5      4     4     4
6      5     2     2
7      5     5     5
8      5     5     5
9      1     1     1
10     6     6     6
11     0     0     0
12     5     6     1
13     1     1     1
14     4     4     4
15     4     4     4
16     6     6     6
17     8     8     8
18     7     7     7
19     2     2     2
20     6     1     6
21     5     5     5
22     4     4     4
23     5     5     5
24     6     6     6
25     7     7     7
26     6     6     6
27     2     2     2
28     8     8     8
29     0     0     0
30     6     6     6
31     1     1     1
32     4     8     4
33     7     7     7
34     4     4     4
35     0     0     0
36     1     1     1
37     1     1     1
38     5     5     5
39     5     5     5
40     1     1     1
41     0     0     0
42     8     8     8
43     6     6     6
44     3     3     3
45     5     5     5
46     0     

In [68]:
#device = "smartphone"
df_monitor_smartphone = df_monitor[df_monitor['device']=='smartphone'].reset_index(drop=True)
calculate_fleiss_kappa(df_monitor_smartphone)

    df_0  df_1  df_2
0      1     1     1
1      3     3     3
2      4     4     4
3      5     5     5
4      1     1     1
5      6     6     6
6      5     6     1
7      1     1     1
8      4     4     4
9      4     4     4
10     6     6     6
11     7     7     7
12     2     2     2
13     6     1     6
14     5     5     5
15     6     6     6
16     8     8     8
17     0     0     0
18     6     6     6
19     7     7     7
20     0     0     0
21     1     1     1
22     5     5     5
23     1     1     1
24     0     0     0
25     6     6     6
26     5     5     5
27     6     6     6
28     0     0     0
29     4     4     4
30     7     1     7
31     4     4     4
32     6     6     6
33     6     6     6
34     1     1     1
35     8     8     3
36     0     0     0
37     4     4     4
38     6     6     6
39     1     1     1
40     0     0     0
41     2     2     2
42     2     2     2
43     6     6     6
44     8     8     8
45     8     8     8
46     2     

In [69]:
#device = "slit"
df_monitor_slit = df_monitor[df_monitor['device']=='slit'].reset_index(drop=True)
calculate_fleiss_kappa(df_monitor_slit)

    df_0  df_1  df_2
0      0     0     0
1      1     1     1
2      4     4     4
3      5     2     2
4      5     5     5
5      0     0     0
6      8     8     8
7      4     4     4
8      5     5     5
9      6     6     6
10     7     7     7
11     2     2     2
12     1     1     1
13     4     8     4
14     4     4     4
15     1     1     1
16     5     5     5
17     8     8     8
18     3     3     3
19     0     0     0
20     2     2     2
21     4     4     4
22     4     4     4
23     1     1     1
24     5     5     5
25     7     7     7
26     5     5     5
27     8     8     8
28     0     0     0
29     6     6     6
30     5     5     5
31     4     4     4
32     1     6     6
33     5     5     5
34     2     2     2
35     0     0     0
36     3     3     3
37     1     1     4
38     7     7     7
39     6     6     6
40     7     7     7
41     6     6     6
42     6     6     6
43     8     8     8
44     2     2     2
45     5     5     5
46     1     